# Closure Objects

This post is inspired by Robert Nystorm's excellent book "Crafting Interpreters".
You can find the original idea in here: https://craftinginterpreters.com/closures.html#challenges

## Closures

What is a closure?
Usually, a function defines its own scope. Local variables that are created inside the function - cease to exist when the function returns.

That "cease to exist" means that objects are no longer reachable and they get garbage-collected.
I would like to show you that objects inside functions don't exist after the function returns - but how can I demonstrate that i _cannot_ access an object anymore?

For that we will use the `weakref` python library, which enables us to hold pointers to objects but doesn't prevent them from getting garbage-collected.
When a weakref points to a collected object, it is dramatically announced "dead".

In [1]:
import gc
import weakref

class TrackedObject:     
    pass

def func():
    global ref
    obj = TrackedObject()
    ref = weakref.ref(obj)
    return

func()
gc.collect()
print(ref)  # <weakref at ...; dead>

<weakref at 0x0000026F99DE5260; dead>


See? dead.

However, functions can "close-over" values from outer scopes - making them live even after the function has returned:

In [2]:
def func():
    global ref
    obj = TrackedObject()
    ref = weakref.ref(obj)
    def inner():
        return obj
    return inner

inner_func = func()
gc.collect()
print(ref)  # <weakref at ...; to 'TrackedObject' at ...>

<weakref at 0x0000026F99DE44A0; to 'TrackedObject' at 0x0000026F99DC51C0>


Now our tracked object is still alive - that's because it is being used by a _closure_ function, which is still accessable.

## Replacing Objects with Closures

In one of the bonus challenges in his book, Robert Nystorm suggests that objects can actually be replaced by closures. He challenges us to implement a contructor to a Vector object, getters and an addition function - without using `class`, only closures.

This is actually pretty cool:

In [3]:
def vector(x,y):
    def get(i):
        return x if i==0 else y
    return get

def get_x(vec):
    return vec(0)
    
def get_y(vec):
    return vec(1)

def vector_add(vec1, vec2):
    return vector(
        get_x(vec1) + get_x(vec2), get_y(vec1) + get_y(vec2)
    )

def print_vector(vec):
    print(f"({get_x(vec)},{get_y(vec)})")

v1 = vector(1, 2)
v2 = vector(4, 7)
v_sum = vector_add(v1, v2)
print_vector(v_sum)  # (5,9)

(5,9)


What else can we do?

The first thing that came to my mind was python's dunder methods. Can we implement addition of closure-objects with the `+` operator?

With 'normal' python objects, we could to that by implementing an `__add__` method in the class.
So, maybe if we add an `__add__` to our closure-objects we could use `+` on them?

Fortunately, because functions in python are objects themselves, we can freely add attributes and methods to them, so that's what we're gonna try (this is actually a bit cheating, because we take advantage of python `function` objects even though this post is about replacing objects with closures. However, we can't change python built-ins and it _will_ be awesome if we could add objects with `+`)

In [4]:
def addable_vector(x,y):
    def get(i):
        return x if i==0 else y
    get.__add__ = vector_add 
    return get

v1 = addable_vector(1, 2)
v2 = addable_vector(4, 7)
v_sum = v1 + v2
print_vector(v_sum)

TypeError: unsupported operand type(s) for +: 'function' and 'function'

That prints the following error:

_TypeError: unsupported operand type(s) for +: 'function' and 'function'_

Well, that happens because when we do `v1 + v2` what python really does is `type(v1).__add__(v2)` rather than call `__add__` on the `v1` instance itself.
`type(v1)` is a `function`, which doesn't have `__add__` so we get an "unsupported" error.

Why don't we add our `__add__` method to `function` then?

In [5]:
def addable_vector(x,y):
    def get(i):
        return x if i==0 else y
    get.__class__.__add__ = vector_add 
    return get

v1 = addable_vector(1, 2)
v2 = addable_vector(4, 7)
v_sum = v1 + v2
print_vector(v_sum)

TypeError: cannot set '__add__' attribute of immutable type 'function'

Now we get:

_TypeError: cannot set '\_\_add\_\_' attribute of immutable type 'function'_

We have reached a dead end - the designers of python have decided that `function` is an immutable type, and therefore we cannot add dunder methods like normal objects.


## Benchmarking

No doubt that defining objects as closures is less readable than using the traditional `class`.
Closure-objects are also more limited than objects because we can't implement dunder methods to them.
The next question we should ask ourselves is - are they more efficient than objects?

Let's start by measuring the size of closure-objects.
For reference, we implement a class version of our vector:

In [6]:
class Vector:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def vector_add(self, other):
        return self.__class__(self.x+other.x, self.y+other.y)

    def print_vector(self):
        print(f"({self.x},{self.y})")

Now let's compare the size in bytes using `sys.getsizeof()`:

In [7]:
import sys

closure_vec = vector(1,2)
object_vec=  Vector(1,2)

print(sys.getsizeof(closure_vec))  # 160
print(sys.getsizeof(object_vec))  # 48

160
48


Wow!

The closure-object is more than *3* times bigger than the regular object!

In [68]:
def some_very_long_name(a,b):
    def inner(i):
        return a if a else b
    return inner

def func2():
    return

class SizedObject:
    pass

co = dir(closure_vec)
o = dir(object_vec)
cl = dir(some_very_long_name(1,2))
f = dir(func2)

print(set(cl) - set(f))
sys.getsizeof(object_vec)

set()


48

In [79]:
many_kwargs = {'a'+str(i):i for i in range(100000)}

def closure_object(**kwargs):
    def get(i):
        return list(kwargs.values())[i]
    return get

class RegularObject:
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

c = closure_object(**many_kwargs)
o = RegularObject(**many_kwargs)
total_size(o)


6647937

In [70]:
def total_size(obj):
    return sum([sys.getsizeof(getattr(obj,attr)) for attr in dir(obj)])

print(total_size(closure_vec))
print(total_size(Vector))
print(total_size(some_very_long_name(1,2)))
print(total_size(some_very_long_name))

12345
2593
12354
12321


In [39]:
attr_name = "__code__"
attr = getattr(closure_vec, attr_name)
print(sys.getsizeof(attr))
for line in attr.co_lines():
    print(line)

216
(0, 2, None)
(2, 4, 2)
(4, 22, 3)


- No doubt this is less readable than objects
- It is probably weaker, in the sense that we can't add dunder methods (class of closure is always `function`)
- What *is* it good for? need to check memory and runtime of this, compared to regular objects

In [ ]:
import sys

v1 = vector(1,2)
sys.getsizeof(v1)

In [ ]:
class Fund:
    def __init__(self, annual_profit):
        self._annual_profit = annual_profit

    @property.getter
    def annual_profit(self):
        return self._annual_profit

    @annual_profit.setter
    def annual_profit(self, _):
        raise AttributeError("Cannot set attribute")


@dataclass
class Fund2:
    annual_profit: float


def fund(annual_profit):
    def inner():
        return annual_profit
    return inner

